In [1]:
import pandas as pd

In [6]:
df = pd.read_csv('data/input/BPI12W.csv')
print(df.head())

# I need to combine rows with the lifecycle:transition "START" and "COMPLETE" (ignoring "SCHEDULED") into single rows with the columns: case_id (case:concept:name), resource (org:resource), activity_name (concept:name), start_timestamp (time:timestamp from the starting event), end_timestamp (time:timestamp from the completing event)

# Handle missing resource values by filling with "unk"
df['org:resource'] = df['org:resource'].fillna('unk')

# Filter for START and COMPLETE events
df_filtered = df[df['lifecycle:transition'].isin(['START', 'COMPLETE'])]
# Pivot the DataFrame to get start and end timestamps
df_pivoted = df_filtered.pivot_table(
    index=['case:concept:name', 'org:resource', 'concept:name'],
    columns='lifecycle:transition',
    values='time:timestamp',
    aggfunc='first'
).reset_index()
# Rename columns for clarity
df_pivoted.columns.name = None  # Remove the name of the columns
df_pivoted.rename(
    columns={
        'START': 'start_timestamp',
        'COMPLETE': 'end_timestamp'
    },
    inplace=True
)
# Drop rows where either start or end timestamp is missing
df_pivoted.dropna(subset=['start_timestamp', 'end_timestamp'], inplace=True)

# Sort the DataFrame by case_id and start_timestamp
df_pivoted.sort_values(
    by=['case:concept:name', 'start_timestamp'],
    inplace=True
)
# Reset index for the final DataFrame
df_final = df_pivoted.reset_index(drop=True)
df_final.rename(
    columns={
        'case:concept:name': 'case_id',
        'org:resource': 'resource',
        'concept:name': 'activity_name'
    },
    inplace=True
)

# Re-order the columns for clarity
df_final = df_final[['case_id', 'resource', 'activity_name', 'start_timestamp', 'end_timestamp']]

# Save the final DataFrame to a CSV file
df_final.to_csv('data/output/BPI12W_processed.csv', index=False)

# Display the first few rows of the final DataFrame
print(df_final.head())

   org:resource lifecycle:transition            concept:name  \
0         112.0             SCHEDULE  W_Completeren aanvraag   
1           NaN                START  W_Completeren aanvraag   
2           NaN             SCHEDULE     W_Nabellen offertes   
3           NaN             COMPLETE  W_Completeren aanvraag   
4           NaN                START     W_Nabellen offertes   

                     time:timestamp                     case:REG_DATE  \
0  2011-10-01 00:39:38.875000+00:00  2011-10-01 00:38:44.546000+00:00   
1  2011-10-01 11:36:46.437000+00:00  2011-10-01 00:38:44.546000+00:00   
2  2011-10-01 11:45:11.554000+00:00  2011-10-01 00:38:44.546000+00:00   
3  2011-10-01 11:45:13.917000+00:00  2011-10-01 00:38:44.546000+00:00   
4  2011-10-01 12:15:41.290000+00:00  2011-10-01 00:38:44.546000+00:00   

   case:concept:name  case:AMOUNT_REQ  
0             173688            20000  
1             173688            20000  
2             173688            20000  
3             17